## Huirong (Sentinel2)

### Pre-flooding

In [1]:
import geemap
import ee

# Initialize the map
Map = geemap.Map()

# Define the point of interest
point = ee.Geometry.Point([129.72635505491021, 42.44534333217912])  

# Get the Sentinel-2 image collection
image = (
    ee.ImageCollection('COPERNICUS/S2')
    .filterBounds(point)
    .filterDate('2008-01-01', '2015-12-31')
    # .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))  # Exclude images with >10% cloud cover
    .sort('CLOUDY_PIXEL_PERCENTAGE')
    .first()
)

# Get the region from the image's geometry
region = image.geometry()

# Define the visualization parameters
# Adjust the min and max values, and select the appropriate bands for RGB visualization
vis_params = {
    'min': 0, 
    'max': 3000, 
    'bands': ['B4', 'B3', 'B2']  # Correct band names for Sentinel-2
}

# Center the map on the region and set the zoom level
Map.centerObject(region, 8)

# Add the image layer to the map
Map.addLayer(image, vis_params, "Sentinel-2 RGB")

# Display the map
# Map


In [2]:
# Define the coordinates for the polygon
coordinates = [
    [129.696351, 42.407487],
    [129.696351, 42.450693],
    [129.797631, 42.450693],
    [129.797631, 42.407487],
    [129.696351, 42.407487]
]

# Create the polygon object
polygon = ee.Geometry.Polygon(coordinates, proj='EPSG:4326', geodesic=False)

training = image.sample(
    **{
        "region": polygon,
        'scale': 1,
        'numPixels': 2000,
        'seed': 0,
        'geometries': True,  # Set this to False to ignore geometries
    }
)

Map.addLayer(training, {}, 'Training samples')
# Map

In [3]:
n_clusters = 9
clusterer = ee.Clusterer.wekaKMeans(n_clusters).train(training)

In [4]:
result = image.cluster(clusterer)
Map.addLayer(result.randomVisualizer(), {}, 'clusters')
Map

Map(center=[41.92651551567432, 129.8825785572717], controls=(WidgetControl(options=['position', 'transparent_b…

여기서부터 일어나서 다시할 것.

#### Export the image

In [5]:
clipped_image_pre = result.clip(polygon)
clipped_image_pre

In [6]:
geemap.ee_export_image(clipped_image_pre, filename='/Users/gigisung/gigi_git/undatathon/results/huirong_pre_unsupervised_clipped.tif', scale=3)


Generating URL ...
Please wait ...
Data downloaded to /Users/gigisung/gigi_git/undatathon/results/huirong_pre_unsupervised_clipped.tif


#### compute each cluster's percentage

In [ ]:
Map=geemap.Map()

# Add the clipped image to the map
Map.addLayer(clipped_image_pre, {}, 'Clipped Image')

# Compute the area of each pixel
pixel_area = ee.Image.pixelArea()

areas = []
for i in range(n_clusters):
    cluster_mask = clipped_image_pre.eq(i)
    cluster_area = cluster_mask.multiply(pixel_area).reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=polygon,
        scale=1,  # Adjust the scale to match the resolution of your image
        bestEffort=True  # Add this line to use the bestEffort option
    )
    info = cluster_area.getInfo()  # Get the information
    areas.append(info['cluster'])  # Use the correct key 'cluster' instead of 'sum'


# Compute the total area
total_area = sum(areas)

# Compute the percentage area for each cluster
percentages = [(area / total_area) * 100 for area in areas]

# Print the results
for i, percentage in enumerate(percentages):
    print(f'Cluster {i}: {percentage:.2f}%')

### Post-flooding

In [7]:
import geemap
import ee

# Initialize the map_post
Map_post = geemap.Map()

# Define the point of interest
point = ee.Geometry.Point([129.72635505491021, 42.44534333217912])  

# Get the Sentinel-2 image_post collection
image_post = (
    ee.ImageCollection('COPERNICUS/S2')
    .filterBounds(point)
    .filterDate('2016-09-03', '2017-05-30')
    # .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))  # Exclude image_posts with >10% cloud cover
    .sort('CLOUDY_PIXEL_PERCENTAGE')
    .first()
)

# Get the region_post from the image_post's geometry
region_post = image_post.geometry()

# Define the visualization parameters
# Adjust the min and max values, and select the appropriate bands for RGB visualization
vis_params = {
    'min': 0, 
    'max': 3000, 
    'bands': ['B4', 'B3', 'B2']  # Correct band names for Sentinel-2
}

# Center the map_post on the region_post and set the zoom level
Map_post.centerObject(region_post, 8)

# Add the image_post layer to the map_post
Map_post.addLayer(image_post, vis_params, "Sentinel-2 RGB")

# Display the map_post
Map_post


Map(center=[42.85657217306809, 129.671592577892], controls=(WidgetControl(options=['position', 'transparent_bg…

In [8]:
geemap.get_info(image_post)

Tree(nodes=(Node(name='Image  (16 bands)', nodes=(Node(icon='file', name='type: Image'), Node(icon='file', nam…

In [9]:
# Define the coordinates for the polygon
coordinates = [
    [129.696351, 42.407487],
    [129.696351, 42.450693],
    [129.797631, 42.450693],
    [129.797631, 42.407487],
    [129.696351, 42.407487]
]

# Create the polygon object
polygon = ee.Geometry.Polygon(coordinates, proj='EPSG:4326', geodesic=False)

# Now use the polygon object as the region argument in your image.sample() call
training_post = image_post.sample(
    **{
        "region": polygon,
        'scale': 1,
        'numPixels': 2000,
        'seed': 0,
        'geometries': True,  # Set this to False to ignore geometries
    }
)

Map_post.addLayer(training_post, {}, 'Training_post samples')
Map_post


Map(bottom=24417.0, center=[42.85657217306809, 129.671592577892], controls=(WidgetControl(options=['position',…

In [10]:
n_clusters_post = 9
clusterer_post = ee.Clusterer.wekaKMeans(n_clusters_post).train(training_post)

In [11]:
result_post = image_post.cluster(clusterer_post)
Map_post.addLayer(result_post.randomVisualizer(), {}, 'clusters')
Map_post

Map(bottom=24417.0, center=[42.85657217306809, 129.671592577892], controls=(WidgetControl(options=['position',…

#### Export the image

In [12]:
result_post

In [13]:
clipped_image = result_post.clip(polygon)
clipped_image

In [14]:
geemap.ee_export_image(clipped_image, filename='results/huirong_post_unsupervised_clipped.tif', scale=3)


Generating URL ...
Please wait ...
Data downloaded to /Users/gigisung/gigi_git/undatathon/results/huirong_post_unsupervised_clipped.tif


### Export to GeoJSON or shp

In [ ]:
from osgeo import gdal, ogr

# Open the raster file
raster = gdal.Open('/Users/gigisung/gigi_git/undatathon/results/huirong_pre_unsupervised_clipped.tif')



In [ ]:
# Get raster geotransform and projection
geotransform = raster.GetGeoTransform()
projection = raster.GetProjection()

# Convert raster to vector
vector = 'results/huirong_post_unsupervised_clipped.shp'
gdal.Polygonize(raster.GetRasterBand(1), None, ogr.GetDriverByName('ESRI Shapefile').CreateDataSource(vector).CreateLayer(vector), -1)


In [ ]:
import geopandas as gpd

# Load the shapefile
gdf = gpd.read_file('/Users/gigisung/gigi_git/undatathon/results/huirong_post_unsupervised_clipped.shp')

# Save as GeoJSON or KML
gdf.to_file('/Users/gigisung/gigi_git/undatathon/results/huirong_post_unsupervised_clipped.geojson', driver='GeoJSON')
# gdf.to_file('output_file.kml', driver='KML')


#### compute each cluster's percentage

In [ ]:
Map_post=geemap.Map()

# Add the clipped image to the map
Map_post.addLayer(clipped_image, {}, 'Clipped Image')

# Compute the area of each pixel
pixel_area_post = ee.Image.pixelArea()

areas = []
for i in range(n_clusters):
    cluster_mask = clipped_image.eq(i)
    cluster_area = cluster_mask.multiply(pixel_area_post).reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=polygon,
        scale=1,  # Adjust the scale to match the resolution of your image
        bestEffort=True  # Add this line to use the bestEffort option
    )
    info = cluster_area.getInfo()  # Get the information
    areas.append(info['cluster'])  # Use the correct key 'cluster' instead of 'sum'


# Compute the total area
total_area_post = sum(areas)

# Compute the percentage area for each cluster
percentages_post = [(area / total_area_post) * 100 for area in areas]

# Print the results
for i, percentage in enumerate(percentages_post):
    print(f'Cluster {i}: {percentage:.2f}%')

### END